# Les Itérateurs, mutabilité, exclusivité et RefCell

---

## Description du problème

- Souvenons-nous de la règle concernant les références mutables qui dit qu'on ne peut accéder à l'objet référence EXCLUSIVEMENT qu'au travers de cette unique référence mutable. Dans le cas de la fonction iter_mut sur `Vec<T>`, qui créer un itérateur mutable, elle prend en paramètre une référence mutable de self, soit de Vec ici.

In [2]:
let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().for_each(|inner| *inner *= 2);
dbg!(&v);

[src/lib.rs:104] &v = [
    2,
    4,
    8,
    16,
]


![ITERMUT](pictures/iter_mut.png)

- Admettons que je tente d'ajouter un élément à mon Vecteur lorsque je suis à l’intérieur du bloc d’itération, **Quel problème pourrait poser ce genre de code ?**

In [3]:
let mut v: Vec<u8> = vec!(1, 2, 4, 8);
v.iter_mut().enumerate().for_each(|(index, inner)| {
    if index == 0 {
        v.push(16);
    }
    *inner *= 2;
});
dbg!(&v);

Error: cannot borrow `v` as mutable more than once at a time

## Solution

### Les raisons du problème

> La création d'un itérateur mutable sur la collection emprunte déjà une référence mutable de cette dernière. Or, à l'intérieur de la closure appeler Vec::push() tente d'emprunter une seconde fois une référence mutable de la collection. Ce qui est interdit.

> Le compilateur ne veut pas, car des effets de bords de tout genre pourraient apparaître, par exemple si j'étends à chaque itération mon vecteur tout en itérant dessus, on aurait ainsi une belle boucle infinie ! Le tout se terminera par un crash du programme dû au fait qu'il n'y ait plus de mémoire disponible !

*Via l'application des règles sur les références, le compilateur protège le code d'erreurs qui sont, pas expériences, extrêmement vicieuses !*

*Il est aussi intéressant de noter qu'en Rust que si j'ai une référence mutable sur un champ d'une structure, je peux avoir une référence mutable sur un autre champ de cette même structure, mais plus sur la structure entière.*

### Un Hack possible avec RefCell ?

- En Rust, il existe un moyen de bypass les règles d'exclusivité des références du compilateur. C'est la création d'une RefCell. Avec frites pates, les règles d'emprunts ne sont plus vérifiées à la compilation, mais durant l'exécution du programme, ainsi si le programmeur sait ce qu'il fait, il peut exécuter un code correctement alors qu'il semblait problématique du point de vue du compilateur.

**RefCell ne fait pas partie de l'unsafe Rust puisque les règles d'emprunt sont toujours vérifiées, mais a l'execution.**

- Reprenons l'exemple au chapitre 3 qui ne compilait pas avec les closures à cause des références mutables, nous utiliserons `RefCell<u8>` au lieu de u8 pour la variable number :

In [14]:
{
    use std::cell::RefCell;                  // RefCell n'est pas dans le prelude, il faut l'importer.
    let number = RefCell::new(1);            // number a pour type RefCell<u8> au lieu de u8 et n'est plus 'mut'
    let mut v: Vec<u8> = vec!(1, 2, 4, 8);
    let ma_closure = |inner: &mut u8| {
        *number.borrow_mut() *= 2;           // borrow_mut() retourne une reference mutable de l'u8 contenu
        *inner *= *number.borrow();          // borrow() retourne une reference non-mutable de l'u8 contenu
    };                                       // On dereference avec * afin de proceder aux operations
    dbg!(number.borrow_mut());               // ICI, IL Y A L'EMPRUNT MUTABLE QUI POSE PROBLEME
    v.iter_mut().for_each(ma_closure);
    dbg!(&v);
    dbg!(number.borrow());
    *number.borrow_mut() *= 2;
    dbg!(number);
}

[src/lib.rs:31] number.borrow_mut() = 1
[src/lib.rs:33] &v = [
    2,
    8,
    32,
    128,
]
[src/lib.rs:34] number.borrow() = 16
[src/lib.rs:36] number = RefCell {
    value: 32,
}


()

> Le code de cet exemple ne semblait donc pas poser de problème puisqu'il vient de s'exécuter correctement. Sans RefCell, le compilateur n'avait pas été en mesure de déterminer que le programme était correct, l'application stricte des règles d'emprunt ne lui a pas permis.

- Mais voici le résultat si on essaie d'utiliser RefCell sur le code qui pose problème dans ce chapitre :

In [5]:
use std::cell::RefCell;
let v: RefCell<Vec<u8>> = RefCell::new(vec!(1, 2, 4, 8));
v.borrow_mut().iter_mut().enumerate().for_each(|(index, inner)| {
    if index == 0 {
        v.borrow_mut().push(16);
    }
    *inner *= 2;
});
dbg!(&v);

thread '<unnamed>' panicked at 'already borrowed: BorrowMutError', src/lib.rs:106:11
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::result::unwrap_failed
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/result.rs:1790:5
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


> OUCH... Le code panique... Cette fois-ci, le compilateur avait raison. Comprenez-vous pourquoi ? :)

*Je vous laisse cela en sujet de réflexion...*